In [3]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 19: Tractor Beam ---</h2><p>Unsure of the state of Santa's ship, you <span title="&quot;borrowed&quot;">borrowed</span> the tractor beam technology from Triton. Time to test it out.</p>
<p>When you're safely away from anything else, you activate the tractor beam, but nothing happens.  It's hard to tell whether it's working if there's nothing to use it on. Fortunately, your ship's drone system can be configured to deploy a drone to specific coordinates and then check whether it's being pulled. There's even an <a href="9">Intcode</a> program (your puzzle input) that gives you access to the drone system.</p>
<p>The program uses two input instructions to request the <em>X and Y position</em> to which the drone should be deployed.  Negative numbers are invalid and will confuse the drone; all numbers should be <em>zero or positive</em>.</p>
<p>Then, the program will output whether the drone is <em>stationary</em> (<code>0</code>) or <em>being pulled by something</em> (<code>1</code>). For example, the coordinate X=<code>0</code>, Y=<code>0</code> is directly in front of the tractor beam emitter, so the drone control program will always report <code>1</code> at that location.</p>
<p>To better understand the tractor beam, it is important to <em>get a good picture</em> of the beam itself. For example, suppose you scan the 10x10 grid of points closest to the emitter:</p>
<pre><code>       X
  0-&gt;      9
 0#.........
 |.#........
 v..##......
  ...###....
  ....###...
Y .....####.
  ......####
  ......####
  .......###
 9........##
</code></pre>
<p>In this example, the <em>number of points affected by the tractor beam</em> in the 10x10 area closest to the emitter is <code><em>27</em></code>.</p>
<p>However, you'll need to scan a larger area to <em>understand the shape</em> of the beam. <em>How many points are affected by the tractor beam in the 50x50 area closest to the emitter?</em> (For each of X and Y, this will be <code>0</code> through <code>49</code>.)</p>
</article>


In [4]:
from IntcodeComputer import IntcodeComputer
from copy import deepcopy


with open("../input/day19.txt") as f:
    puzzle = f.read()


ic = IntcodeComputer(puzzle)

n = 50
count = 0
for y in range(n):
    for x in range(n):
        itr = deepcopy(ic).run()
        itr.send(None)
        itr.send(x)
        if itr.send(y) == 1:
            print("#", end="")
            count += 1
        else:
            print(".", end="")

    print()
print()

print(f"Part I: {count}")

#.................................................
..................................................
..................................................
..#...............................................
...#..............................................
...#..............................................
....#.............................................
.....#............................................
.....##...........................................
......#...........................................
......##..........................................
.......##.........................................
........##........................................
........##........................................
.........##.......................................
.........###......................................
..........###.....................................
...........##.....................................
...........###....................................
............###................

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>206</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>You aren't sure how large Santa's ship is. You aren't even sure if you'll need to use this thing on Santa's ship, but it doesn't hurt to be prepared. You figure Santa's ship might fit in a <em>100x100</em> square.</p>
<p>The beam gets wider as it travels away from the emitter; you'll need to be a minimum distance away to fit a square of that size into the beam fully. (Don't rotate the square; it should be aligned to the same axes as the drone grid.)</p>
<p>For example, suppose you have the following tractor beam readings:</p>
<pre><code>#.......................................
.#......................................
..##....................................
...###..................................
....###.................................
.....####...............................
......#####.............................
......######............................
.......#######..........................
........########........................
.........#########......................
..........#########.....................
...........##########...................
...........############.................
............############................
.............#############..............
..............##############............
...............###############..........
................###############.........
................#################.......
.................########<em>O</em>OOOOOOOOO.....
..................#######OOOOOOOOOO#....
...................######OOOOOOOOOO###..
....................#####OOOOOOOOOO#####
.....................####OOOOOOOOOO#####
.....................####OOOOOOOOOO#####
......................###OOOOOOOOOO#####
.......................##OOOOOOOOOO#####
........................#OOOOOOOOOO#####
.........................OOOOOOOOOO#####
..........................##############
..........................##############
...........................#############
............................############
.............................###########
</code></pre>
<p>In this example, the <em>10x10</em> square closest to the emitter that fits entirely within the tractor beam has been marked <code>O</code>. Within it, the point closest to the emitter (the only highlighted <code><em>O</em></code>) is at X=<code>25</code>, Y=<code>20</code>.</p>
<p>Find the <em>100x100</em> square closest to the emitter that fits entirely within the tractor beam; within that square, find the point closest to the emitter.  <em>What value do you get if you take that point's X coordinate, multiply it by <code>10000</code>, then add the point's Y coordinate?</em> (In the example above, this would be <code>250020</code>.)</p>
</article>

</main>


In [5]:
# # BruteForce
#  Way too slow!!!!!!!!!!!
#
# ic = IntcodeComputer(puzzle)

# n = 1_000
# m = 100
# count = 0
# grid = []
# for y in range(n):
#     grid.append([])
#     for x in range(n):
#         itr = deepcopy(ic).run()
#         itr.send(None)
#         itr.send(x)
#         answer = itr.send(y)
#         if answer == 1:
#             grid[-1].append("#")
#             count += 1
#         elif answer == 0:
#             grid[-1].append(".")


# def check(x, y):
#     for dy in range(m):
#         for dx in range(m):
#             if grid[y + dy][x + dx] != "#":
#                 return False
#     return True


# def set_O(x, y):
#     for dy in range(m):
#         for dx in range(m):
#             grid[y + dy][x + dx] = "O"


# def find():
#     for y in range(n - m):
#         for x in range(n - m):
#             if check(x, y):
#                 set_O(x, y)
#                 return x, y
#     return -1, -1


# # print("\n".join("".join(l) for l in grid))
# x, y = find()
# print("   " + "".join(f"{f'{i // 10}' if i % 10 == 0 else ' '}" for i in range(n)))
# print("   " + "".join(f"{i % 10}" for i in range(n)))
# print(f"\n".join(f"{i:2d} " + "".join(l) for i, l in enumerate(grid)))
# print("   " + "".join(f"{i % 10}" for i in range(n)))
# print("   " + "".join(f"{f'{i // 10}' if i % 10 == 0 else ' '}" for i in range(n)))


# print(f"Part II: {x*10_000 + y}")

In [6]:
def get_status(ic: IntcodeComputer, x: int, y: int) -> int:
    itr = deepcopy(ic).run()
    itr.send(None)
    itr.send(x)
    answer = itr.send(y)
    return answer


def partII():
    ic = IntcodeComputer(puzzle)

    x, y = 0, 0
    while not get_status(ic, x + 99, y):
        y += 1
        while not get_status(ic, x, y + 99):
            x += 1
    return x * 10000 + y


print(f"Part II: {partII()}")

Part II: 6190948


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>6190948</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
